In [1]:
import json
import re
import pandas as pd

In [2]:

# Opening JSON file
def EvaluateWorldVote(record):
    try:
        gamedate= record['date']
        whoname= record['from']
        whoid= record['from_id']
        performance =ExtractPerformance(record['text'])
        return {
            'gamedate': gamedate,
            'whoname': whoname,
            'whoid': whoid,
            'gamenumber': performance['gamenumber'],
            'performance': performance['performance']
        }
    except:
        return {
        'gamedate': '',
        'whoname': '',
        'whoid': '',
        'gamenumber': '',
        'performance': ''
    }


In [3]:

def ExtractPerformance(text):
    try:
        vote = EvalTextSplitArray(text)
        gamenumber= re.findall('[#]\d+', vote)
        performance = re.findall('[\w|\d]\/\d', vote)
        if gamenumber:
            gamenumber = gamenumber[0].replace('#','')
        if performance:
            performance = performance[0].split('/')
            if 'x' in str(performance[0]).lower():
                performance[0] = str(performance[0]).lower().replace('x','7')

        return {
            'gamenumber':gamenumber,
            'performance': performance[0]
        }
    except:
        return {
        'gamenumber':'',
        'performance': ''
    }


In [4]:

def EvalTextSplitArray(text):
    if len(text) > 1:
        for palabra in text:
            if "palabra del día" in palabra:
                wordlvote = palabra.split('\n')
                for vote in wordlvote:
                    if "palabra del día" in vote:
                        return vote
    return ""


Aca ejecuta los metodos

In [10]:
jsonFile = open('result.json')
TelegramConversations = json.load(jsonFile)

parcialresults = pd.DataFrame(columns=['gamedate', 'whoname', 'whoid', 'gamenumber','performance'])

for record in TelegramConversations['messages']:
    texto = str(record['text'])
    if "La palabra del día" in texto:
        vote=EvaluateWorldVote(record)
        votedf = pd.DataFrame([vote])
        parcialresults = pd.concat([parcialresults,votedf],  ignore_index=True, sort=False)
            # df = df.append(vote, ignore_index = True)

parcialresults.to_csv('results.csv', index=False)
jsonFile.close()



In [11]:
parcialresults

,gamedate,whoname,whoid,gamenumber,performance
0,2022-10-05T10:29:08,Gustavo Talavan,user36307305,272,3
1,2022-10-05T11:01:59,Daniel Modini,user286600771,272,2
2,2022-10-05T17:57:14,Ariel Mantovani,user29274611,272,2
3,2022-10-06T09:30:43,Gustavo Talavan,user36307305,273,3
4,2022-10-06T09:36:05,Ariel Mantovani,user29274611,273,3
...,...,...,...,...,...
1399,2023-12-21T23:21:50,Nicolas Lopez barrios,user167977758,714,2
1400,2023-12-22T07:46:18,Daniel Modini,user286600771,715,4
1401,2023-12-22T08:26:14,Ariel Mantovani,user29274611,715,4
1402,2023-12-22T08:29:36,Nicolas Lopez barrios,user167977758,715,4
